<a href="https://colab.research.google.com/github/afeawoL/Cpp.Playground/blob/master/Resume_parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Challenge 1:** Resume Analyzer

*The task description has been disected into task sections(roman numerals) for example (i), (ii) etc.*


##Task:
Process a dataset of resumes*(i)* and automatically score*(ii)* their fit to the following job announcements based on content similarity.
  
Justify the score*(iii)* (0-100) based on how well the content in the applicants resume matches the job application requirements.
in the req.

Use a transformer-based model/LLM downloaded and run locally on CPU-only (max 32G RAM)*(iv)*.

Breakdown
*(i)* Input: Resume PDF file(s) → Convert: Parsable Text Format → *(ii)* Output: Similiarity Score(%) + *(iii)* Justification *(Utilizing highlighted words/phrases)*



Resumes shall be located in {WORKSPACE NAME}/bah_resume_detector/data/resumes/

In [3]:
!python --version       #Verifying python version meets requirements
!which python           #Verify where python is installed
!ls /content/bah_resume_detector/ #Confirm resume data is readable

Python 3.10.12
/usr/local/bin/python
data  output




```
# `HF_TOKEN` needs to be set in Secrets. For the purposes of this notebook we shall utilize public HF repos but you should set HF_TOKEN if private.
```



In [26]:
!pip install --quiet transformers torch sentence-transformers pypdf beautifulsoup4       #Installing required packages


In [27]:
#Set directory paths

import os #Utilize os functions
import torch #
from pypdf import PdfReader #PDF to text extractor


#Resume and Job directory paths
resume_dirs = '/content/bah_resume_detector/data/resumes/'
job_description_dirs = '/content/bah_resume_detector/data/job_descriptions/'

#Extract text from resume pdfs
def extract_resume(resume_dirs):
  resume_text= ""
  with open(file_path, "rb") as file:
        pdf_reader = PdfReader(file)
        for page in pdf_reader.pages:
            text += page.extract_text() + "\n"
  return resume_text

#Extract text from job description pdfs
def extract_job_description(job_description_dirs):
  job_description_text= ""
  with open(file_path, "rb") as file:
        pdf_reader = PdfReader(file)
        for page in pdf_reader.pages:
            text += page.extract_text() + "\n"
  return job_description_text


In [30]:
#Preload text data for comparison

def load_resume_data(resume_dirs):  #Load Resumes
    resume_data = []
    for filename in os.listdir(resume_dirs):
        if filename.endswith('.pdf'):
            file_path = os.path.join(resume_dirs, filename)
            resume_text = extract_resume(file_path)
            resume_data.append(resume_text)
    return resume_data

def load_job_description_data(job_description_dirs):  #Load Job Descriptions
    job_description_data = []
    for filename in os.listdir(job_description_dirs):
        if filename.endswith('.pdf'):
            file_path = os.path.join(job_description_dirs, filename)
            job_description_text = extract_job_description(file_path)
            job_description_data.append(job_description_text)
    return job_description_data

def load_data(resume_dirs, job_description_dirs):
    resume_data = load_resume_data(resume_dirs)
    job_description_data = load_job_description_data(job_description_dirs)
    return resume_data, job_description_data


In [34]:
#Similiarity Scoring
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer, util

#HF MODEL TO USE sentence-transformers/multi-qa-mpnet-base-cos-v1
nlp_model= SentenceTransformer('sentence-transformers/multi-qa-mpnet-base-cos-v1')

#Calculate scores
def score_resumes(resume_data, job_description_data):
    scores = []
    for resume_text in resume_data:
        resume_embedding = nlp_model.encode(resume_text, convert_to_tensor=True)
        for job_description_text in job_description_data:
            job_description_embedding = nlp_model.encode(job_description_text, convert_to_tensor=True)
            similarity_score = util.pytorch_cos_sim(resume_embedding, job_description_embedding).item()
            scores.append(similarity_score)
    return scores

def main():
    resume_data, job_description_data = load_data(resume_dirs, job_description_dirs)
    scores = score_resumes(resume_data, job_description_data)
    return scores

if __name__ == "__main__":
    main()



RuntimeError: Failed to import transformers.integrations.integration_utils because of the following error (look up to see its traceback):
Failed to import transformers.modeling_utils because of the following error (look up to see its traceback):
No module named 'torch._custom_ops'

In [ ]:
def analyze_resumes(scores):
    result=[]




#**References**


PyPDF2:
* https://pypi.org/project/PyPDF2/


spaCY:
* https://spacy.io/usage/spacy-101
* https://www.analyticsvidhya.com/blog/2020/03/spacy-tutorial-learn-natural-language-processing/

Microsoft BitNet:
*   https://colab.research.google.com/drive/1ovmQUOtnYIdvcBkwEE4MzVL1HKfFHdNT?usp=sharing
* https://huggingface.co/blog/1_58_llm_extreme_quantization

Sentence Transformer pretrained models
* https://sbert.net/docs/sentence_transformer/pretrained_models.html

